# Predicting Rainfall Anomalies in the Amazon
## Model Outline:
- Preprocessing
- Multivariate EOF
- Ridge Regression
- Lasso Regression
- Neural Network Regression

Created by David Gotler, 2021

## Preprocessing
### Define a dataframe class to:
- Load in the datasets:
    - feature dataset (ERA5): variables are CAPE, CIN, geopotential, relative humidity
        - lat: (0,-30), lon: (-40,-80)
    - target dataset (GPCC): variable is precipitation
        - lat: (-5,-25), lon: (-50,-70)
- Detrend the dataset by removing linear trends. We want the dataset to be stationary
- Calculate standardized anomalies by subtracting the monthly mean and dividing by standard deviation. We want centered dataset with unit variance.

In [35]:
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
    
from dataframe import dataframe

# directories
data_dir = 'data_store/'
features_fname = 'ERA5.features.1950-2019.nc'
target_fname = 'precip.mon.total.1x1.v2020.nc'

# Create df object to store our feature and target datasets
df = dataframe(data_dir,features_fname,target_fname)
df.detrend()
df.std_anom()
df.remove_nan()
df.flatten()

/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/Users/davidgotler/opt/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:166: RankWarnin

In [37]:
from forecast import DirectForecaster,RecursiveForecaster
forecaster = DirectForecaster(LinearRegression(),lags=3,target_months=[9,10,11],steps=3,include_autoreg=True,avg_features=False)
X_train,y_train,X_test,y_test = forecaster.train_test_split([2016,2017,2018,2019],df.target_da,df.features_da)
forecaster.fit(X_train,y_train)
y_pred = forecaster.predict(X_test,y_test)
print('mean absolute error on the test set: {:.3f}%'.format(mean_absolute_error(y_test,y_pred)*100))


<xarray.DataArray 'sample' (sample: 37)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36])
Dimensions without coordinates: sample


ValueError: Could not convert object to NumPy datetime